#Trabalho da disciplina de ciência de dados
---
## 1. dataset
A base de dados foi retirada deste link:
https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016
## 2. objetivos do trabalho
O objetivo do trabalho é fazer uma análise de dados sobre um conjunto de dados de suicícios que ocorreram entre os anos de 1985 à 2016.

##3. Descrição da base de dados:

A base de dados junta todos os dados de suicídios registrados que ocorreram entre os anos 1985 à 2016 como país de origem, sexo, faixa etária, renda etc. A tabela possui 27820 linhas x 12 colunas.

### vamos começar importando todos as bibliotecas necessárias

In [ ]:
# Instalação do Dash
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table

In [2]:
# Pacotes para criação de processos para suportar o serviço URL externo do Ngrok
import os.path
import sys, json
import requests
import subprocess

# Pacotes de tratamento de URL
from requests.exceptions import RequestException
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from collections import namedtuple

In [3]:
# Definição da função de Download do Ngrok
def download_ngrok():
    if not os.path.isfile('ngrok'):
        !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
        !unzip -o ngrok-stable-linux-amd64.zip
    pass

In [4]:
# Criação de um Túnel entre uma aplicação no Colab e uma URL externa no Ngrok
# Função para pegar a referência nesse túnel

Response = namedtuple('Response', ['url', 'error'])

def get_tunnel():
    try:
        Tunnel = subprocess.Popen(['./ngrok','http','8050'])

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)

        res = session.get('http://localhost:4040/api/tunnels')
        res.raise_for_status()

        tunnel_str = res.text
        tunnel_cfg = json.loads(tunnel_str)
        tunnel_url = tunnel_cfg['tunnels'][0]['public_url']

        return Response(url=tunnel_url, error=None)
    except RequestException as e:
        return Response(url=None, error=str(e))

In [30]:
%%writefile dash1.py
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# MUNDO
tabela = pd.read_csv('/content/master.csv')

tabela2 = tabela.groupby(by=['country','year'], as_index=False)["suicides_no"].sum()

tabela3 = tabela.groupby(by='country', as_index=False)['suicides_no'].sum() 

group1 = tabela.groupby(by=['sex'], as_index=False)["suicides_no"].sum()

group2 = tabela.groupby(by=['age'], as_index=False)["suicides_no"].sum()

group3 = tabela.groupby(by=['generation'], as_index=False)["suicides_no"].sum()

my_map = px.scatter_geo(tabela2, 
                        locations='country',
                        locationmode = 'country names',
                        size = 'suicides_no',
                        animation_frame = 'year',
                        projection='natural earth',
                        color = 'country',
                        title = 'Suicide number over the years in the world')

my_map2 = px.scatter_geo(tabela3, 
                        locations='country',
                        locationmode = 'country names',
                        size = 'suicides_no',
                        projection='natural earth',
                        color = 'country',
                        title = 'Total suicide numbers in the world')

pie_1 = px.pie(group1, values='suicides_no', names='sex', color='sex', 
               title='Gráfico de seção mostrando como o número total de suicídos em função do sexo')

pie_2 = px.pie(group2, values='suicides_no', names='age', color='age', 
               title='Gráfico de seção mostrando como o número total de suicídos em função da faixa etária')

pie_3 = px.pie(group3, values='suicides_no', names='generation', color='generation', 
               title='Gráfico de seção mostrando como o número total de suicídos em função da geração')

# TABELA E GRAFICOS BRAZIL
tabela_brazil = tabela.loc[tabela['country']=='Brazil']

tabela_brazil = tabela_brazil.loc[tabela_brazil.index.repeat(tabela_brazil['suicides_no'])].reset_index(drop=True)

fig1 = px.histogram(tabela_brazil,x ='sex',color='sex',
                    title='Proporção de suicídios no Brasil de acordo com o sexo')

fig3 = px.histogram(tabela_brazil,x ='age',color='sex',
                    title='Proporção de suicídios no Brasil de acordo com a faixa etária')

fig5 = px.histogram(tabela_brazil,x ='year',color='year',
                    title='Suicícos no Brasil ao longo dos anos')

# TABELA E GRAFICOS RUSSIA
tabela_USA = tabela.loc[tabela['country']=='Russian Federation']
tabela_USA = tabela_USA.loc[tabela_USA.index.repeat(tabela_USA['suicides_no'])].reset_index(drop=True)

fig2 = px.histogram(tabela_USA,x ='sex',color='sex',
                    title='Proporção de suicídios no Brasil de acordo com o sexo')

fig4 = px.histogram(tabela_USA,x ='age',color='sex',
                    title='Proporção de suicídios no Brasil de acordo com a faixa etária')

fig6 = px.histogram(tabela_USA,x ='year',color='year',
                    title='Suicídos na Russia ao longo dos anos')

intro = '''
### Introdução
O trabalho tem como objetivo fazer uma análise de dados dos suicidos 
registrado entre os anos de 1985 até 2016.
'''
world = '''
### No mundo
Os gráficos à seguir mostra dados referente ao mundo inteiro.
'''
Russia = '''
### Na Russia
Os gráficos à seguir são referentes a Russia, que possui o maior 
número de suicídios registrados entre todos os países do mundo, aproximamente 15% do total.
'''
brazil = '''
### No Brasil
'''

app.layout = html.Div(children=[
    html.H1(
        children='Data Analisis about Suicides',
        style={
            'textAlign': 'center',
            #'color': colors['text']
        }),

    dcc.Markdown(children=intro),

    dcc.Markdown(children=world),
    
    html.Div(children=[
      dcc.Graph(
          id='pie_1',
          figure=pie_1,
      ),
      dcc.Graph(
          id='pie_2',
          figure=pie_2,
      ),
      dcc.Graph(
          id='pie_3',
          figure=pie_3,
      )
    ]),
  
    html.Div(children=[
      dcc.Graph(
          id='my_map',
          figure=my_map
      ),
      dcc.Graph(
          id='my_map2',
          figure=my_map2
      ),
    ]),

    dcc.Markdown(children=Russia),

    html.Div(children=[
      dcc.Graph(
          id='hist_1',
          figure=fig2,
      ),
      dcc.Graph(
          id='hist_2',
          figure=fig4,
      ),
      dcc.Graph(
          id='hist_3',
          figure=fig6,
      )
    ]),

    dcc.Markdown(children=brazil),

    html.Div(children=[
      dcc.Graph(
          id='hist_4',
          figure=fig1,
      ),
      dcc.Graph(
          id='hist_5',
          figure=fig3,
      ),
      dcc.Graph(
          id='hist_6',
          figure=fig5,
      )
    ]),
  
    
])

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting dash1.py


In [ ]:
download_ngrok()

In [ ]:
retorno = get_tunnel()
print(retorno)  # Para podermos saber o endereço da URL da Aplicação
!python dash1.py